In [1]:
from dotenv import load_dotenv
from rich import print as rprint


load_dotenv()

True

## 도구 생성
- Type Hints 필수 : tool의 입력 스키마 정의
- Docstring : 모델이 tool의 목적을 이해하는데 도움이 되도록 간결하고 직관적으로 작성

### 1. 가장 간단한 방법 : `@tool`

In [8]:
from langchain.tools import tool

@tool
# @tool("custom_search", description="Search the customer database for records matching the query.") # Custom Tool의 이름
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.
    Args:
        query: Search terms to look for
        limit: Maximum number of results of return
    """
    return f"Found {limit} results for '{query}'"

In [9]:
rprint(search_database) 

StructuredTool(
    name='search_database',
    description='Search the customer database for records matching the query.\n    Args:\n        query: Search 
terms to look for\n        limit: Maximum number of results of return',
    args_schema=<class 'langchain_core.utils.pydantic.search_database'>,
    func=<function search_database at 0x762bc6fbfd80>
)

#### 스키마 정의
- Pydantic
- Json

In [15]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """
    Input for weather queries.
    """
    location: str = Field(description="City name or coordinates")
    units: Literal["celesius", "fahrenheit"] = Field(default="celesius", description="Temperature unit preference")
    include_forecast: bool = Field(default=False, description="Include 5-day forecast")


@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celesius", include_forecast: bool = False) -> str:
    """
    Get current weather and optional forecast.
    """
    temp = 22 if units == "celesius" else 72
    result = f"Current weather in {location}: {temp}°{units}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [16]:
rprint(get_weather)

StructuredTool(
    name='get_weather',
    description='Get current weather and optional forecast.',
    args_schema=<class '__main__.WeatherInput'>,
    func=<function get_weather at 0x762bc522c0e0>
)

## 컨텍스트 접근
- 도구는 에이전트 상태, 런타임 컨텍스트 및 장기 메모리에 접근할 수 있을 때 강력.
- 도구는 컨텍스트 인식 결정을 내리고, 응답을 개인화하며, 대화 전반에 걸쳐 정보 유지 가능

In [17]:
from langchain.tools import tool, ToolRuntime

# Access the current conversation state
@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

In [18]:
rprint(summarize_conversation)
rprint(get_user_preference)

StructuredTool(
    name='summarize_conversation',
    description='Summarize the conversation so far.',
    args_schema=<class 'langchain_core.utils.pydantic.summarize_conversation'>,
    func=<function summarize_conversation at 0x762bc522dd00>
)

StructuredTool(
    name='get_user_preference',
    description='Get a user preference value.',
    args_schema=<class 'langchain_core.utils.pydantic.get_user_preference'>,
    func=<function get_user_preference at 0x762bc522da80>
)

음.. 뭔가 바로바로 확인이 안되서 사용하기 좀 그런데